# BHAI VAE: Semi-Supervised Variational Autoencoder for Ocean Drilling Data

This notebook implements the complete analysis pipeline for the BHAI VAE paper:

1. **Data Pipeline**: Download from Zenodo, process to 20cm bins
2. **Descriptive Figures**: Expedition map, dataset overview, lithology counts, variable distributions
3. **Model Training**: Unsupervised and Semi-supervised VAE with masked encoding
4. **Model Evaluation**: SVM classification, clustering metrics (ARI), reconstruction quality
5. **Evaluation Figures**: Reconstruction scatter, ROC comparison, UMAP visualization
6. **Network Diagrams**: Architecture visualization from PyTorch specs

## 1. Setup and Imports

In [ ]:
# Core libraries
import os
import sys
import random
import warnings
from pathlib import Path
from datetime import datetime

# Data
import numpy as np
import pandas as pd

# Deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (r2_score, roc_curve, auc, accuracy_score, 
                             precision_recall_fscore_support, adjusted_rand_score,
                             classification_report, confusion_matrix)
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cluster import KMeans

# UMAP
import umap

# Progress
from tqdm.auto import tqdm

# Download
import requests
import zipfile

# Mapping
try:
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    HAS_CARTOPY = True
except ImportError:
    HAS_CARTOPY = False
    print("Warning: cartopy not available, expedition map will be skipped")

warnings.filterwarnings('ignore')

# Add models directory to path
sys.path.insert(0, str(Path.cwd().parent))
from models.vae import VAE, SemiSupervisedVAE, DistributionAwareScaler, FEATURE_COLS

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Configuration

In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================

# Reproducibility
SEED = 42

# Paths
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
PROCESSED_DATA_DIR = DATA_DIR / "processed"
MODEL_DIR = BASE_DIR / "models"
FIGURES_DIR = BASE_DIR / "figures"

# Create directories
for d in [RAW_DATA_DIR, PROCESSED_DATA_DIR, MODEL_DIR, FIGURES_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Data source
ZENODO_DOI = "10.5281/zenodo.7484524"
ZENODO_RECORD_ID = "7484524"

# Model architecture
INPUT_DIM = 6
LATENT_DIM = 10
HIDDEN_DIMS = [64, 32]

# Training hyperparameters
N_EPOCHS = 100
BATCH_SIZE = 256
LEARNING_RATE = 1e-3
MASK_RATIO = 0.10  # 10% masked encoding

# Annealing schedules
BETA_START = 1e-10
BETA_END = 0.075
BETA_ANNEAL_EPOCHS = 50

ALPHA_START = 0.0
ALPHA_END = 0.01
ALPHA_ANNEAL_EPOCHS = 50

# Device
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Feature columns
FEATURE_COLUMNS = [
    "Bulk density (GRA)",
    "Magnetic susceptibility (instr. units)",
    "NGR total counts (cps)",
    "R", "G", "B"
]

FEATURE_NAMES_SHORT = ['Bulk Density', 'Mag. Susc.', 'NGR', 'R', 'G', 'B']

print(f"Configuration loaded")
print(f"  Device: {DEVICE}")
print(f"  Base dir: {BASE_DIR}")
print(f"  Epochs: {N_EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Mask ratio: {MASK_RATIO}")

In [ ]:
# Set random seeds for reproducibility
def set_seed(seed=SEED):
    """Set all random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(SEED)
print(f"Random seed set to {SEED}")

## 3. Data Download from Zenodo

In [ ]:
def download_from_zenodo(record_id, output_dir, files_to_download=None):
    """
    Download files from Zenodo record.
    
    Parameters
    ----------
    record_id : str
        Zenodo record ID (e.g., "7484524")
    output_dir : Path
        Directory to save downloaded files
    files_to_download : list, optional
        Specific files to download. If None, downloads all files.
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Get record metadata
    api_url = f"https://zenodo.org/api/records/{record_id}"
    response = requests.get(api_url)
    response.raise_for_status()
    record = response.json()
    
    print(f"Record: {record['metadata']['title']}")
    print(f"DOI: {record['doi']}")
    print(f"Files available: {len(record['files'])}")
    
    downloaded = []
    for file_info in record['files']:
        filename = file_info['key']
        
        # Filter files if specified
        if files_to_download and filename not in files_to_download:
            continue
            
        output_path = output_dir / filename
        
        # Skip if already exists
        if output_path.exists():
            print(f"  {filename} already exists, skipping")
            downloaded.append(output_path)
            continue
        
        # Download
        print(f"  Downloading {filename} ({file_info['size'] / 1e6:.1f} MB)...")
        download_url = file_info['links']['self']
        
        with requests.get(download_url, stream=True) as r:
            r.raise_for_status()
            total_size = int(r.headers.get('content-length', 0))
            
            with open(output_path, 'wb') as f:
                with tqdm(total=total_size, unit='B', unit_scale=True, desc=filename) as pbar:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
                        pbar.update(len(chunk))
        
        downloaded.append(output_path)
        print(f"    Saved to {output_path}")
    
    return downloaded

In [ ]:
# Files we need from Zenodo
ZENODO_FILES = [
    "GRA_DataLITH.csv",
    "MS_DataLITH.csv",
    "NGR_DataLITH.csv",
    "RSC_DataLITH.csv"  # Contains L*a*b* and RGB
]

# Check if raw data exists, download if not
raw_files_exist = all((RAW_DATA_DIR / f).exists() for f in ZENODO_FILES)

if not raw_files_exist:
    print("Downloading raw data from Zenodo...")
    downloaded = download_from_zenodo(ZENODO_RECORD_ID, RAW_DATA_DIR, ZENODO_FILES)
    print(f"\nDownloaded {len(downloaded)} files")
else:
    print("Raw data already exists in", RAW_DATA_DIR)
    for f in ZENODO_FILES:
        path = RAW_DATA_DIR / f
        if path.exists():
            print(f"  {f}: {path.stat().st_size / 1e6:.1f} MB")

## 4. Data Processing

In [ ]:
def process_raw_data(raw_dir, output_path, bin_size_cm=20):
    """
    Process raw LILY data files into training format.
    
    Steps:
    1. Load each measurement type
    2. Create depth bins (default 20cm)
    3. Aggregate measurements within bins (mean)
    4. Inner join all measurements
    5. Filter for complete samples
    
    Parameters
    ----------
    raw_dir : Path
        Directory containing raw CSV files
    output_path : Path
        Output path for processed CSV
    bin_size_cm : int
        Depth bin size in centimeters
    """
    raw_dir = Path(raw_dir)
    bin_size_m = bin_size_cm / 100.0
    
    print(f"Processing raw data with {bin_size_cm}cm bins...")
    
    # Load GRA (Bulk Density)
    print("  Loading GRA...")
    gra = pd.read_csv(raw_dir / "GRA_DataLITH.csv")
    gra['Depth_Bin'] = (gra['Top depth CSF-A (m)'] // bin_size_m) * bin_size_m
    gra_agg = gra.groupby(['Borehole_ID', 'Depth_Bin']).agg({
        'Bulk density (GRA)': 'mean',
        'Principal': 'first'
    }).reset_index()
    
    # Load MS (Magnetic Susceptibility)
    print("  Loading MS...")
    ms = pd.read_csv(raw_dir / "MS_DataLITH.csv")
    ms['Depth_Bin'] = (ms['Top depth CSF-A (m)'] // bin_size_m) * bin_size_m
    ms_agg = ms.groupby(['Borehole_ID', 'Depth_Bin']).agg({
        'Magnetic susceptibility (instr. units)': 'mean'
    }).reset_index()
    
    # Load NGR
    print("  Loading NGR...")
    ngr = pd.read_csv(raw_dir / "NGR_DataLITH.csv")
    ngr['Depth_Bin'] = (ngr['Top depth CSF-A (m)'] // bin_size_m) * bin_size_m
    ngr_agg = ngr.groupby(['Borehole_ID', 'Depth_Bin']).agg({
        'NGR total counts (cps)': 'mean'
    }).reset_index()
    
    # Load RSC (RGB/L*a*b*)
    print("  Loading RSC...")
    rsc = pd.read_csv(raw_dir / "RSC_DataLITH.csv")
    rsc['Depth_Bin'] = (rsc['Top depth CSF-A (m)'] // bin_size_m) * bin_size_m
    rsc_agg = rsc.groupby(['Borehole_ID', 'Depth_Bin']).agg({
        'R': 'mean', 'G': 'mean', 'B': 'mean'
    }).reset_index()
    
    # Merge all datasets (inner join for complete samples)
    print("  Merging datasets...")
    merged = gra_agg.merge(ms_agg, on=['Borehole_ID', 'Depth_Bin'], how='inner')
    merged = merged.merge(ngr_agg, on=['Borehole_ID', 'Depth_Bin'], how='inner')
    merged = merged.merge(rsc_agg, on=['Borehole_ID', 'Depth_Bin'], how='inner')
    
    # Remove rows with any NaN in features
    feature_cols = ['Bulk density (GRA)', 'Magnetic susceptibility (instr. units)',
                   'NGR total counts (cps)', 'R', 'G', 'B']
    before = len(merged)
    merged = merged.dropna(subset=feature_cols + ['Principal'])
    after = len(merged)
    
    print(f"  Samples: {before} -> {after} (dropped {before - after} incomplete)")
    
    # Save
    merged.to_csv(output_path, index=False)
    print(f"  Saved to {output_path}")
    
    return merged

In [ ]:
# Use existing processed data or process from raw
PROCESSED_DATA_PATH = DATA_DIR / "vae_training_data_v2_20cm.csv"

if PROCESSED_DATA_PATH.exists():
    print(f"Loading existing processed data from {PROCESSED_DATA_PATH}")
    df = pd.read_csv(PROCESSED_DATA_PATH)
else:
    print("Processing raw data...")
    df = process_raw_data(RAW_DATA_DIR, PROCESSED_DATA_PATH, bin_size_cm=20)

print(f"\nDataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nSample counts by lithology (top 10):")
print(df['Principal'].value_counts().head(10))

In [ ]:
# Prepare data for training
print("Preparing data for training...")

# Extract features
X_raw = df[FEATURE_COLUMNS].values
print(f"Features shape: {X_raw.shape}")

# Scale using DistributionAwareScaler
scaler = DistributionAwareScaler()
X_scaled = scaler.fit_transform(X_raw)
print(f"Scaled features shape: {X_scaled.shape}")

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Principal'].values)
n_classes = len(label_encoder.classes_)
print(f"Number of lithology classes: {n_classes}")

# Extract expedition info
df['Expedition'] = df['Borehole_ID'].str.extract(r'^(\d+)')[0].astype(int)
expeditions = df['Expedition'].unique()
print(f"Number of expeditions: {len(expeditions)}")
print(f"Expeditions: {sorted(expeditions)}")

## 5. Descriptive Figures

### 5.1 Expedition Map

In [ ]:
# IODP expedition coordinates (approximate central locations)
# Based on IODP expedition reports
EXPEDITION_COORDS = {
    346: (40.5, 138.5),   # Asian Monsoon (Sea of Japan)
    349: (15.5, 117.0),   # South China Sea
    350: (32.0, 139.5),   # Izu-Bonin-Mariana
    351: (30.5, 134.5),   # Izu-Bonin-Mariana
    352: (28.5, 141.0),   # Izu-Bonin-Mariana
    353: (8.0, 87.0),     # iMonsoon
    354: (8.0, 88.5),     # Bengal Fan
    355: (22.0, 68.0),    # Arabian Sea Monsoon
    356: (-28.0, 113.0),  # Indonesian Throughflow
    359: (5.0, 73.0),     # Maldives Monsoon and Sea Level
    360: (-42.0, 9.0),    # SW Indian Ridge
    361: (-41.5, -8.0),   # S African Climates
    362: (-2.0, 102.0),   # Sumatra Seismogenesis
    363: (-5.0, 142.0),   # W Pacific Warm Pool
    366: (13.5, 147.0),   # Mariana Convergent Margin
    367: (18.5, 116.5),   # South China Sea Rifted Margin
    368: (19.0, 116.0),   # South China Sea Rifted Margin
    "368X": (19.5, 115.5), # South China Sea Rifted Margin Extension
    369: (-33.5, 104.0),  # Australia Cretaceous Climate
    371: (-34.0, 163.5),  # Tasman Frontier
    372: (-35.0, 174.0),  # Creeping Gas Hydrate Slides
    374: (-77.0, 175.0),  # Ross Sea West Antarctic Ice Sheet
    375: (-39.0, 178.5),  # Hikurangi Subduction Margin
    376: (-37.5, 177.0),  # Brothers Arc Flux
    379: (-57.5, -92.5),  # Amundsen Sea West Antarctic Ice Sheet
}

if HAS_CARTOPY:
    # Count samples per expedition
    exp_counts = df.groupby('Expedition').size().to_dict()
    
    # Create figure
    fig = plt.figure(figsize=(16, 9))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson(central_longitude=150))
    
    # Add map features
    ax.set_global()
    ax.add_feature(cfeature.LAND, facecolor='#f5f5dc', edgecolor='gray', linewidth=0.5)
    ax.add_feature(cfeature.OCEAN, facecolor='#add8e6')
    ax.gridlines(draw_labels=False, linewidth=0.5, alpha=0.5)
    
    # Plot expeditions
    for exp, (lat, lon) in EXPEDITION_COORDS.items():
        exp_int = int(exp) if isinstance(exp, (int, float)) else exp
        count = exp_counts.get(exp_int, exp_counts.get(str(exp), 0))
        
        # Size based on sample count
        size = np.sqrt(count) * 2 + 50
        
        ax.scatter(lon, lat, s=size, c='#cd5c5c', alpha=0.7,
                  transform=ccrs.PlateCarree(), zorder=5, edgecolors='darkred', linewidth=0.5)
        
        # Label
        ax.text(lon + 3, lat + 2, str(exp), fontsize=8,
               transform=ccrs.PlateCarree(), zorder=6)
    
    # Legend
    legend_sizes = [1000, 10000, 40000]
    legend_labels = ['1,000', '10,000', '40,000']
    legend_handles = [plt.scatter([], [], s=np.sqrt(s)*2+50, c='#cd5c5c', 
                                  edgecolors='darkred', linewidth=0.5, alpha=0.7)
                     for s in legend_sizes]
    
    legend = ax.legend(legend_handles, legend_labels, 
                      title='Samples', loc='upper right',
                      frameon=True, fontsize=9)
    
    ax.set_title('IODP Boreholes from the LILY Dataset', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'fig_lily_expedition_map.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"Saved: {FIGURES_DIR / 'fig_lily_expedition_map.png'}")
else:
    print("Skipping expedition map (cartopy not available)")

### 5.2 Dataset Overview

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Panel 1: Samples per expedition
ax = axes[0, 0]
exp_counts = df['Expedition'].value_counts().sort_index()
bars = ax.bar(exp_counts.index.astype(str), exp_counts.values, color='steelblue', edgecolor='black', alpha=0.7)
ax.set_xlabel('Expedition', fontsize=11)
ax.set_ylabel('Number of Samples', fontsize=11)
ax.set_title('Samples per Expedition', fontsize=12, fontweight='bold')
ax.tick_params(axis='x', rotation=45)
ax.grid(axis='y', alpha=0.3)

# Panel 2: Boreholes per expedition
ax = axes[0, 1]
borehole_counts = df.groupby('Expedition')['Borehole_ID'].nunique().sort_index()
ax.bar(borehole_counts.index.astype(str), borehole_counts.values, color='coral', edgecolor='black', alpha=0.7)
ax.set_xlabel('Expedition', fontsize=11)
ax.set_ylabel('Number of Boreholes', fontsize=11)
ax.set_title('Boreholes per Expedition', fontsize=12, fontweight='bold')
ax.tick_params(axis='x', rotation=45)
ax.grid(axis='y', alpha=0.3)

# Panel 3: Depth distribution
ax = axes[1, 0]
ax.hist(df['Depth_Bin'], bins=100, color='seagreen', edgecolor='black', alpha=0.7)
ax.set_xlabel('Depth (m)', fontsize=11)
ax.set_ylabel('Frequency', fontsize=11)
ax.set_title('Depth Distribution', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Panel 4: Summary statistics text
ax = axes[1, 1]
ax.axis('off')

summary_text = f"""
Dataset Summary
{'='*40}

Total samples: {len(df):,}
Total boreholes: {df['Borehole_ID'].nunique():,}
Total expeditions: {df['Expedition'].nunique()}
Lithology classes: {n_classes}

Depth range: {df['Depth_Bin'].min():.1f} - {df['Depth_Bin'].max():.1f} m
Mean depth: {df['Depth_Bin'].mean():.1f} m

Features:
  • Bulk density (GRA)
  • Magnetic susceptibility
  • Natural gamma radiation (NGR)
  • RGB color (R, G, B)

Bin size: 20 cm
"""
ax.text(0.1, 0.9, summary_text, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig_lily_dataset.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'fig_lily_dataset.png'}")

### 5.3 Lithology Class Counts

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

# Get top 30 lithology classes
lith_counts = df['Principal'].value_counts().head(30)

# Color by frequency tier
colors = []
for count in lith_counts.values:
    if count > 10000:
        colors.append('#1f77b4')  # Blue for most common
    elif count > 5000:
        colors.append('#2ca02c')  # Green
    elif count > 1000:
        colors.append('#ff7f0e')  # Orange
    else:
        colors.append('#d62728')  # Red for rare

bars = ax.barh(range(len(lith_counts)), lith_counts.values, color=colors, edgecolor='black', alpha=0.8)
ax.set_yticks(range(len(lith_counts)))
ax.set_yticklabels(lith_counts.index, fontsize=9)
ax.invert_yaxis()
ax.set_xlabel('Number of Samples', fontsize=12)
ax.set_title('Top 30 Lithology Classes by Sample Count', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add count labels
for i, (count, bar) in enumerate(zip(lith_counts.values, bars)):
    ax.text(count + 200, i, f'{count:,}', va='center', fontsize=8)

# Legend for colors
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#1f77b4', edgecolor='black', label='>10,000'),
    Patch(facecolor='#2ca02c', edgecolor='black', label='5,000-10,000'),
    Patch(facecolor='#ff7f0e', edgecolor='black', label='1,000-5,000'),
    Patch(facecolor='#d62728', edgecolor='black', label='<1,000'),
]
ax.legend(handles=legend_elements, title='Sample Count', loc='lower right')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig_lily_lithology_counts.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'fig_lily_lithology_counts.png'}")

### 5.4 Variable Distributions

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

for i, (col, name, color) in enumerate(zip(FEATURE_COLUMNS, FEATURE_NAMES_SHORT, colors)):
    ax = axes[i]
    
    data = df[col].dropna()
    
    # Histogram
    ax.hist(data, bins=100, color=color, edgecolor='black', alpha=0.7, density=True)
    
    # Add statistics
    stats_text = f'μ={data.mean():.2f}\nσ={data.std():.2f}\nn={len(data):,}'
    ax.text(0.95, 0.95, stats_text, transform=ax.transAxes, fontsize=9,
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.set_xlabel(name, fontsize=11)
    ax.set_ylabel('Density', fontsize=11)
    ax.set_title(f'{name} Distribution', fontsize=12, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig_lily_variables_dist.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'fig_lily_variables_dist.png'}")

## 6. Model Definitions

Models are imported from `models/vae.py`. Here we verify the architecture.

In [ ]:
# Verify model architectures
print("Unsupervised VAE Architecture:")
print("="*50)
model_unsup_test = VAE(input_dim=INPUT_DIM, latent_dim=LATENT_DIM, hidden_dims=HIDDEN_DIMS)
print(model_unsup_test)
print(f"\nTotal parameters: {sum(p.numel() for p in model_unsup_test.parameters()):,}")

print("\n" + "="*50)
print("Semi-Supervised VAE Architecture:")
print("="*50)
model_semisup_test = SemiSupervisedVAE(input_dim=INPUT_DIM, latent_dim=LATENT_DIM, 
                                        hidden_dims=HIDDEN_DIMS, n_classes=n_classes)
print(model_semisup_test)
print(f"\nTotal parameters: {sum(p.numel() for p in model_semisup_test.parameters()):,}")

## 7. Training Utilities

In [ ]:
def get_beta(epoch, beta_start=BETA_START, beta_end=BETA_END, anneal_epochs=BETA_ANNEAL_EPOCHS):
    """Get beta value for current epoch (linear annealing)."""
    if epoch >= anneal_epochs:
        return beta_end
    return beta_start + (beta_end - beta_start) * (epoch / anneal_epochs)

def get_alpha(epoch, alpha_start=ALPHA_START, alpha_end=ALPHA_END, anneal_epochs=ALPHA_ANNEAL_EPOCHS):
    """Get alpha value for current epoch (linear annealing)."""
    if epoch >= anneal_epochs:
        return alpha_end
    return alpha_start + (alpha_end - alpha_start) * (epoch / anneal_epochs)

def apply_mask(x, mask_ratio=MASK_RATIO):
    """
    Apply random masking to input tensor.
    
    Randomly sets mask_ratio fraction of values to 0.
    Returns masked tensor and mask.
    """
    mask = torch.rand_like(x) < mask_ratio
    x_masked = x.clone()
    x_masked[mask] = 0
    return x_masked, mask

# Visualize annealing schedules
epochs = np.arange(N_EPOCHS)
betas = [get_beta(e) for e in epochs]
alphas = [get_alpha(e) for e in epochs]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

ax = axes[0]
ax.plot(epochs, betas, 'b-', linewidth=2)
ax.axhline(y=BETA_END, color='r', linestyle='--', alpha=0.5, label=f'β_end = {BETA_END}')
ax.axvline(x=BETA_ANNEAL_EPOCHS, color='g', linestyle='--', alpha=0.5, label=f'Anneal end = {BETA_ANNEAL_EPOCHS}')
ax.set_xlabel('Epoch')
ax.set_ylabel('β (KL weight)')
ax.set_title('β-Annealing Schedule')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1]
ax.plot(epochs, alphas, 'orange', linewidth=2)
ax.axhline(y=ALPHA_END, color='r', linestyle='--', alpha=0.5, label=f'α_end = {ALPHA_END}')
ax.axvline(x=ALPHA_ANNEAL_EPOCHS, color='g', linestyle='--', alpha=0.5, label=f'Anneal end = {ALPHA_ANNEAL_EPOCHS}')
ax.set_xlabel('Epoch')
ax.set_ylabel('α (Classification weight)')
ax.set_title('α-Annealing Schedule (Semi-supervised)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Train Unsupervised VAE

In [ ]:
def train_unsupervised_vae(X, n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, 
                           mask_ratio=MASK_RATIO, device=DEVICE):
    """
    Train unsupervised VAE with masked encoding and β-annealing.
    
    Parameters
    ----------
    X : np.ndarray
        Scaled input features
    n_epochs : int
        Number of training epochs
    batch_size : int
        Batch size
    mask_ratio : float
        Fraction of inputs to mask during training
    device : torch.device
        Device to train on
    
    Returns
    -------
    model : VAE
        Trained model
    history : dict
        Training history
    """
    set_seed(SEED)
    
    # Create dataset and dataloader
    dataset = TensorDataset(torch.FloatTensor(X))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                           num_workers=0, pin_memory=True)
    
    # Initialize model
    model = VAE(input_dim=INPUT_DIM, latent_dim=LATENT_DIM, hidden_dims=HIDDEN_DIMS).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Training history
    history = {'loss': [], 'recon_loss': [], 'kl_loss': [], 'beta': []}
    
    # Training loop
    pbar = tqdm(range(n_epochs), desc='Training Unsup VAE')
    for epoch in pbar:
        model.train()
        epoch_loss = 0
        epoch_recon = 0
        epoch_kl = 0
        n_batches = 0
        
        beta = get_beta(epoch)
        
        for batch in dataloader:
            x_batch = batch[0].to(device)
            
            # Apply masking
            x_masked, mask = apply_mask(x_batch, mask_ratio)
            
            # Forward pass
            optimizer.zero_grad()
            recon, mu, logvar = model(x_masked)
            
            # Loss
            loss, recon_loss, kl_loss = model.loss_function(recon, x_batch, mu, logvar, beta)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
            epoch_recon += recon_loss.item()
            epoch_kl += kl_loss.item()
            n_batches += 1
        
        # Record history
        history['loss'].append(epoch_loss / n_batches)
        history['recon_loss'].append(epoch_recon / n_batches)
        history['kl_loss'].append(epoch_kl / n_batches)
        history['beta'].append(beta)
        
        # Update progress bar
        pbar.set_postfix({
            'loss': f"{epoch_loss/n_batches:.1f}",
            'recon': f"{epoch_recon/n_batches:.1f}",
            'kl': f"{epoch_kl/n_batches:.1f}",
            'β': f"{beta:.2e}"
        })
    
    return model, history

In [ ]:
# Train unsupervised VAE
print(f"Training unsupervised VAE on {len(X_scaled):,} samples...")
print(f"  Device: {DEVICE}")
print(f"  Epochs: {N_EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Mask ratio: {MASK_RATIO}")
print(f"  β-annealing: {BETA_START} → {BETA_END} over {BETA_ANNEAL_EPOCHS} epochs")
print()

model_unsup, history_unsup = train_unsupervised_vae(X_scaled)

# Save model
torch.save(model_unsup.state_dict(), MODEL_DIR / 'model_unsup.pt')
print(f"\nModel saved to {MODEL_DIR / 'model_unsup.pt'}")

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

ax = axes[0]
ax.plot(history_unsup['loss'], 'b-', linewidth=1.5)
ax.set_xlabel('Epoch')
ax.set_ylabel('Total Loss')
ax.set_title('Unsupervised VAE: Total Loss')
ax.grid(True, alpha=0.3)

ax = axes[1]
ax.plot(history_unsup['recon_loss'], 'g-', linewidth=1.5, label='Recon Loss')
ax.set_xlabel('Epoch')
ax.set_ylabel('Reconstruction Loss')
ax.set_title('Reconstruction Loss')
ax.grid(True, alpha=0.3)

ax = axes[2]
ax.plot(history_unsup['kl_loss'], 'r-', linewidth=1.5, label='KL Loss')
ax2 = ax.twinx()
ax2.plot(history_unsup['beta'], 'b--', linewidth=1, alpha=0.7, label='β')
ax.set_xlabel('Epoch')
ax.set_ylabel('KL Loss', color='r')
ax2.set_ylabel('β', color='b')
ax.set_title('KL Loss and β Schedule')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Train Semi-Supervised VAE

In [ ]:
def train_semisupervised_vae(X, y, n_epochs=N_EPOCHS, batch_size=BATCH_SIZE,
                              mask_ratio=MASK_RATIO, device=DEVICE):
    """
    Train semi-supervised VAE with masked encoding, β-annealing, and α-annealing.
    
    Parameters
    ----------
    X : np.ndarray
        Scaled input features
    y : np.ndarray
        Encoded labels
    n_epochs : int
        Number of training epochs
    batch_size : int
        Batch size
    mask_ratio : float
        Fraction of inputs to mask during training
    device : torch.device
        Device to train on
    
    Returns
    -------
    model : SemiSupervisedVAE
        Trained model
    history : dict
        Training history
    """
    set_seed(SEED)
    
    n_classes = len(np.unique(y))
    
    # Create dataset and dataloader
    dataset = TensorDataset(torch.FloatTensor(X), torch.LongTensor(y))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True,
                           num_workers=0, pin_memory=True)
    
    # Initialize model
    model = SemiSupervisedVAE(input_dim=INPUT_DIM, latent_dim=LATENT_DIM,
                              hidden_dims=HIDDEN_DIMS, n_classes=n_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Training history
    history = {'loss': [], 'recon_loss': [], 'kl_loss': [], 'class_loss': [], 
               'beta': [], 'alpha': [], 'accuracy': []}
    
    # Training loop
    pbar = tqdm(range(n_epochs), desc='Training Semi-sup VAE')
    for epoch in pbar:
        model.train()
        epoch_loss = 0
        epoch_recon = 0
        epoch_kl = 0
        epoch_class = 0
        epoch_correct = 0
        epoch_total = 0
        n_batches = 0
        
        beta = get_beta(epoch)
        alpha = get_alpha(epoch)
        
        for batch in dataloader:
            x_batch, y_batch = batch
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            
            # Apply masking
            x_masked, mask = apply_mask(x_batch, mask_ratio)
            
            # Forward pass
            optimizer.zero_grad()
            recon, mu, logvar, logits = model(x_masked)
            
            # Loss
            loss, recon_loss, kl_loss, class_loss = model.loss_function(
                recon, x_batch, mu, logvar, logits, y_batch, beta, alpha
            )
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            # Accuracy
            _, predicted = torch.max(logits, 1)
            epoch_correct += (predicted == y_batch).sum().item()
            epoch_total += y_batch.size(0)
            
            epoch_loss += loss.item()
            epoch_recon += recon_loss.item()
            epoch_kl += kl_loss.item()
            epoch_class += class_loss.item()
            n_batches += 1
        
        # Record history
        history['loss'].append(epoch_loss / n_batches)
        history['recon_loss'].append(epoch_recon / n_batches)
        history['kl_loss'].append(epoch_kl / n_batches)
        history['class_loss'].append(epoch_class / n_batches)
        history['beta'].append(beta)
        history['alpha'].append(alpha)
        history['accuracy'].append(epoch_correct / epoch_total)
        
        # Update progress bar
        pbar.set_postfix({
            'loss': f"{epoch_loss/n_batches:.1f}",
            'acc': f"{epoch_correct/epoch_total:.3f}",
            'β': f"{beta:.2e}",
            'α': f"{alpha:.3f}"
        })
    
    return model, history

In [ ]:
# Train semi-supervised VAE
print(f"Training semi-supervised VAE on {len(X_scaled):,} samples...")
print(f"  Device: {DEVICE}")
print(f"  Classes: {n_classes}")
print(f"  Epochs: {N_EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Mask ratio: {MASK_RATIO}")
print(f"  β-annealing: {BETA_START} → {BETA_END} over {BETA_ANNEAL_EPOCHS} epochs")
print(f"  α-annealing: {ALPHA_START} → {ALPHA_END} over {ALPHA_ANNEAL_EPOCHS} epochs")
print()

model_semisup, history_semisup = train_semisupervised_vae(X_scaled, y)

# Save model
torch.save(model_semisup.state_dict(), MODEL_DIR / 'model_semisup.pt')
print(f"\nModel saved to {MODEL_DIR / 'model_semisup.pt'}")

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

ax = axes[0, 0]
ax.plot(history_semisup['loss'], 'b-', linewidth=1.5)
ax.set_xlabel('Epoch')
ax.set_ylabel('Total Loss')
ax.set_title('Semi-supervised VAE: Total Loss')
ax.grid(True, alpha=0.3)

ax = axes[0, 1]
ax.plot(history_semisup['recon_loss'], 'g-', linewidth=1.5)
ax.set_xlabel('Epoch')
ax.set_ylabel('Reconstruction Loss')
ax.set_title('Reconstruction Loss')
ax.grid(True, alpha=0.3)

ax = axes[0, 2]
ax.plot(history_semisup['kl_loss'], 'r-', linewidth=1.5)
ax2 = ax.twinx()
ax2.plot(history_semisup['beta'], 'b--', linewidth=1, alpha=0.7)
ax.set_xlabel('Epoch')
ax.set_ylabel('KL Loss', color='r')
ax2.set_ylabel('β', color='b')
ax.set_title('KL Loss and β Schedule')
ax.grid(True, alpha=0.3)

ax = axes[1, 0]
ax.plot(history_semisup['class_loss'], 'purple', linewidth=1.5)
ax2 = ax.twinx()
ax2.plot(history_semisup['alpha'], 'orange', linestyle='--', linewidth=1, alpha=0.7)
ax.set_xlabel('Epoch')
ax.set_ylabel('Classification Loss', color='purple')
ax2.set_ylabel('α', color='orange')
ax.set_title('Classification Loss and α Schedule')
ax.grid(True, alpha=0.3)

ax = axes[1, 1]
ax.plot(history_semisup['accuracy'], 'teal', linewidth=1.5)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('Training Accuracy')
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)

# Compare both models
ax = axes[1, 2]
ax.plot(history_unsup['recon_loss'], 'b-', linewidth=1.5, label='Unsupervised')
ax.plot(history_semisup['recon_loss'], 'r-', linewidth=1.5, label='Semi-supervised')
ax.set_xlabel('Epoch')
ax.set_ylabel('Reconstruction Loss')
ax.set_title('Reconstruction Loss Comparison')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Evaluation Utilities

In [ ]:
def get_embeddings(model, X, device=DEVICE):
    """Extract embeddings from model."""
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X).to(device)
        embeddings = model.get_embeddings(X_tensor).cpu().numpy()
    return embeddings

def get_reconstructions(model, X, device=DEVICE):
    """Get reconstructions using deterministic encoding (mu)."""
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X).to(device)
        mu, _ = model.encode(X_tensor)
        recon = model.decode(mu)
    return recon.cpu().numpy()

def get_predictions(model, X, device=DEVICE):
    """Get class predictions from semi-supervised model."""
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X).to(device)
        mu, _ = model.encode(X_tensor)
        logits = model.classify(mu)
        probs = F.softmax(logits, dim=1)
        _, preds = torch.max(logits, 1)
    return preds.cpu().numpy(), probs.cpu().numpy()

def compute_ari(embeddings, y, k_values=[10, 12, 15, 20]):
    """Compute Adjusted Rand Index for k-means clustering."""
    results = {}
    for k in k_values:
        kmeans = KMeans(n_clusters=k, random_state=SEED, n_init=10)
        clusters = kmeans.fit_predict(embeddings)
        ari = adjusted_rand_score(y, clusters)
        results[k] = ari
    return results

## 11. Evaluate Models

In [ ]:
# Extract embeddings and reconstructions
print("Extracting embeddings and reconstructions...")

emb_unsup = get_embeddings(model_unsup, X_scaled)
emb_semisup = get_embeddings(model_semisup, X_scaled)

recon_unsup = get_reconstructions(model_unsup, X_scaled)
recon_semisup = get_reconstructions(model_semisup, X_scaled)

print(f"  Unsupervised embeddings: {emb_unsup.shape}")
print(f"  Semi-supervised embeddings: {emb_semisup.shape}")

# Save embeddings
np.savez(MODEL_DIR / 'embeddings.npz',
         unsup=emb_unsup, semisup=emb_semisup, y=y)
print(f"\nEmbeddings saved to {MODEL_DIR / 'embeddings.npz'}")

In [ ]:
# Reconstruction quality (R²)
print("\nReconstruction Quality (R²):")
print("="*50)

r2_results = {'Feature': [], 'Unsupervised': [], 'Semi-supervised': []}

for i, name in enumerate(FEATURE_NAMES_SHORT):
    r2_unsup = r2_score(X_scaled[:, i], recon_unsup[:, i])
    r2_semisup = r2_score(X_scaled[:, i], recon_semisup[:, i])
    
    r2_results['Feature'].append(name)
    r2_results['Unsupervised'].append(r2_unsup)
    r2_results['Semi-supervised'].append(r2_semisup)
    
    print(f"  {name:15s}: Unsup R² = {r2_unsup:.4f}, Semi-sup R² = {r2_semisup:.4f}")

# Overall
r2_overall_unsup = r2_score(X_scaled.flatten(), recon_unsup.flatten())
r2_overall_semisup = r2_score(X_scaled.flatten(), recon_semisup.flatten())
print(f"\n  {'Overall':15s}: Unsup R² = {r2_overall_unsup:.4f}, Semi-sup R² = {r2_overall_semisup:.4f}")

r2_df = pd.DataFrame(r2_results)
r2_df

In [ ]:
# Clustering metrics (ARI)
print("\nClustering Quality (ARI):")
print("="*50)

ari_unsup = compute_ari(emb_unsup, y)
ari_semisup = compute_ari(emb_semisup, y)

print("\nUnsupervised VAE:")
for k, ari in ari_unsup.items():
    print(f"  k={k}: ARI = {ari:.4f}")

print("\nSemi-supervised VAE:")
for k, ari in ari_semisup.items():
    print(f"  k={k}: ARI = {ari:.4f}")

In [ ]:
# SVM classification on unsupervised embeddings
print("\nSVM Classification on Unsupervised Embeddings:")
print("="*50)

# Use subset for faster SVM training, filtering for classes with >=2 samples
n_svm_samples = min(50000, len(y))
svm_idx = np.random.choice(len(y), n_svm_samples, replace=False)

# Filter to keep only classes with >=2 samples (required for stratified split)
class_counts_svm = pd.Series(y[svm_idx]).value_counts()
valid_classes = class_counts_svm[class_counts_svm >= 2].index.values
valid_mask = np.isin(y[svm_idx], valid_classes)
svm_idx_filtered = svm_idx[valid_mask]

print(f"Filtered from {len(svm_idx):,} to {len(svm_idx_filtered):,} samples")
print(f"(keeping classes with >=2 samples for stratified split)")

# Train/test split
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(
    emb_unsup[svm_idx_filtered], y[svm_idx_filtered], test_size=0.2, 
    random_state=SEED, stratify=y[svm_idx_filtered]
)

print(f"Training SVM on {len(X_train_svm):,} samples...")
svm = SVC(kernel='rbf', C=1.0, random_state=SEED)
svm.fit(X_train_svm, y_train_svm)

y_pred_svm = svm.predict(X_test_svm)
acc_svm = accuracy_score(y_test_svm, y_pred_svm)
precision, recall, f1, _ = precision_recall_fscore_support(y_test_svm, y_pred_svm, average='weighted')

print(f"\nSVM Results (on unsupervised embeddings):")
print(f"  Accuracy:  {acc_svm:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1:        {f1:.4f}")

In [ ]:
# Semi-supervised classification head evaluation
print("\nSemi-supervised Classification Head Evaluation:")
print("="*50)

y_pred_semisup, y_prob_semisup = get_predictions(model_semisup, X_scaled)

acc_semisup = accuracy_score(y, y_pred_semisup)
precision_ss, recall_ss, f1_ss, _ = precision_recall_fscore_support(y, y_pred_semisup, average='weighted')

print(f"\nClassification Head Results:")
print(f"  Accuracy:  {acc_semisup:.4f}")
print(f"  Precision: {precision_ss:.4f}")
print(f"  Recall:    {recall_ss:.4f}")
print(f"  F1:        {f1_ss:.4f}")

In [ ]:
# Summary comparison
print("\n" + "="*70)
print("MODEL COMPARISON SUMMARY")
print("="*70)

comparison_data = {
    'Metric': ['R² (overall)', 'Accuracy', 'Precision', 'Recall', 'F1', f'ARI (k={list(ari_unsup.keys())[0]})'],
    'Unsupervised (SVM)': [r2_overall_unsup, acc_svm, precision, recall, f1, list(ari_unsup.values())[0]],
    'Semi-supervised': [r2_overall_semisup, acc_semisup, precision_ss, recall_ss, f1_ss, list(ari_semisup.values())[0]]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

## 12. Evaluation Figures

### 12.1 Reconstruction Scatter Plot

In [ ]:
# Inverse transform to original scale for visualization
X_orig_raw = scaler.inverse_transform(X_scaled)
X_unsup_raw = scaler.inverse_transform(recon_unsup)
X_semisup_raw = scaler.inverse_transform(recon_semisup)

fig, axes = plt.subplots(2, 6, figsize=(18, 8))

for i, name in enumerate(FEATURE_NAMES_SHORT):
    # Unsupervised (top row)
    ax = axes[0, i]
    r2 = r2_score(X_orig_raw[:, i], X_unsup_raw[:, i])
    ax.scatter(X_orig_raw[:, i], X_unsup_raw[:, i], alpha=0.05, s=0.5, rasterized=True)
    
    # Reference line
    lims = [min(X_orig_raw[:, i].min(), X_unsup_raw[:, i].min()),
            max(X_orig_raw[:, i].max(), X_unsup_raw[:, i].max())]
    ax.plot(lims, lims, 'r--', lw=2)
    
    ax.set_xlabel(f'True {name}', fontsize=9)
    if i == 0:
        ax.set_ylabel('Unsupervised\nPredicted', fontsize=10)
    ax.set_title(f'R²={r2:.3f}', fontsize=10)
    
    # Semi-supervised (bottom row)
    ax = axes[1, i]
    r2 = r2_score(X_orig_raw[:, i], X_semisup_raw[:, i])
    ax.scatter(X_orig_raw[:, i], X_semisup_raw[:, i], alpha=0.05, s=0.5, rasterized=True, c='orange')
    
    lims = [min(X_orig_raw[:, i].min(), X_semisup_raw[:, i].min()),
            max(X_orig_raw[:, i].max(), X_semisup_raw[:, i].max())]
    ax.plot(lims, lims, 'r--', lw=2)
    
    ax.set_xlabel(f'True {name}', fontsize=9)
    if i == 0:
        ax.set_ylabel('Semi-supervised\nPredicted', fontsize=10)
    ax.set_title(f'R²={r2:.3f}', fontsize=10)

plt.suptitle('Reconstruction Quality: True vs Predicted Values', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig_reconstruction_scatter.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'fig_reconstruction_scatter.png'}")

### 12.2 ROC Comparison

In [ ]:
# Get top 10 classes for ROC curve visualization
class_counts = np.bincount(y)
top10_classes = np.argsort(class_counts)[-10:]

# Subsample for computational efficiency
n_roc_samples = min(20000, len(y))
roc_idx = np.random.choice(len(y), n_roc_samples, replace=False)

# Filter to top 10 classes
mask = np.isin(y[roc_idx], top10_classes)
y_roc = y[roc_idx][mask]
emb_unsup_roc = emb_unsup[roc_idx][mask]
emb_semisup_roc = emb_semisup[roc_idx][mask]

# Binarize labels
y_bin = label_binarize(y_roc, classes=top10_classes)

print(f"ROC analysis on {len(y_roc):,} samples from top 10 classes")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
colors = plt.cm.tab10(np.linspace(0, 1, 10))

# Unsupervised (SVM)
print("Training SVM for unsupervised ROC...")
svm_roc = OneVsRestClassifier(SVC(kernel='rbf', probability=True, random_state=SEED))
svm_roc.fit(emb_unsup_roc, y_bin)
y_score_unsup = svm_roc.predict_proba(emb_unsup_roc)

ax = axes[0]
mean_auc_unsup = 0
for i, cls in enumerate(top10_classes):
    fpr, tpr, _ = roc_curve(y_bin[:, i], y_score_unsup[:, i])
    roc_auc = auc(fpr, tpr)
    mean_auc_unsup += roc_auc
    name = label_encoder.inverse_transform([cls])[0][:15]
    ax.plot(fpr, tpr, lw=2, alpha=0.8, color=colors[i], label=f'{name} ({roc_auc:.2f})')

ax.plot([0, 1], [0, 1], 'k--', lw=1, alpha=0.5)
ax.set_xlabel('False Positive Rate', fontsize=11)
ax.set_ylabel('True Positive Rate', fontsize=11)
ax.set_title(f'Unsupervised VAE + SVM (mean AUC={mean_auc_unsup/10:.2f})', fontsize=12)
ax.legend(loc='lower right', fontsize=8)
ax.grid(True, alpha=0.3)

# Semi-supervised (classification head)
print("Computing semi-supervised ROC...")
_, y_prob_roc = get_predictions(model_semisup, X_scaled[roc_idx][mask])
y_score_semisup = y_prob_roc[:, top10_classes]

ax = axes[1]
mean_auc_semisup = 0
for i, cls in enumerate(top10_classes):
    fpr, tpr, _ = roc_curve(y_bin[:, i], y_score_semisup[:, i])
    roc_auc = auc(fpr, tpr)
    mean_auc_semisup += roc_auc
    name = label_encoder.inverse_transform([cls])[0][:15]
    ax.plot(fpr, tpr, lw=2, alpha=0.8, color=colors[i], label=f'{name} ({roc_auc:.2f})')

ax.plot([0, 1], [0, 1], 'k--', lw=1, alpha=0.5)
ax.set_xlabel('False Positive Rate', fontsize=11)
ax.set_ylabel('True Positive Rate', fontsize=11)
ax.set_title(f'Semi-supervised VAE (mean AUC={mean_auc_semisup/10:.2f})', fontsize=12)
ax.legend(loc='lower right', fontsize=8)
ax.grid(True, alpha=0.3)

plt.suptitle('ROC Curves: Top 10 Lithology Classes', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig_roc_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'fig_roc_comparison.png'}")

### 12.3 R² Comparison Plot

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

r2_unsup_list = r2_df['Unsupervised'].values
r2_semisup_list = r2_df['Semi-supervised'].values
features = r2_df['Feature'].values

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

for i, (r2_u, r2_s, name, color) in enumerate(zip(r2_unsup_list, r2_semisup_list, features, colors)):
    ax.scatter(r2_u, r2_s, s=200, c=color, label=name, edgecolors='black', linewidth=1.5, zorder=5)

# Diagonal line
lims = [min(min(r2_unsup_list), min(r2_semisup_list)) - 0.02,
        max(max(r2_unsup_list), max(r2_semisup_list)) + 0.02]
ax.plot(lims, lims, 'k--', lw=2, alpha=0.5, label='Equal performance')

ax.set_xlabel('R² Unsupervised VAE', fontsize=12)
ax.set_ylabel('R² Semi-supervised VAE', fontsize=12)
ax.set_title('Reconstruction Quality Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.set_aspect('equal')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig_r2_unsup_vs_semi.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'fig_r2_unsup_vs_semi.png'}")

### 12.4 UMAP Visualization

In [ ]:
# UMAP embedding visualization
print("Computing UMAP embeddings (this may take a few minutes)...")

# Subsample for UMAP
n_umap_samples = min(50000, len(y))
umap_idx = np.random.choice(len(y), n_umap_samples, replace=False)

# Get top 15 classes for coloring
top15_classes = set(np.argsort(class_counts)[-15:])

print(f"Computing UMAP for {n_umap_samples:,} samples...")

# UMAP for unsupervised
print("  Unsupervised embeddings...")
reducer_unsup = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=SEED, n_jobs=-1)
umap_unsup = reducer_unsup.fit_transform(emb_unsup[umap_idx])

# UMAP for semi-supervised
print("  Semi-supervised embeddings...")
reducer_semisup = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=SEED, n_jobs=-1)
umap_semisup = reducer_semisup.fit_transform(emb_semisup[umap_idx])

print("  Done!")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

y_sub = y[umap_idx]

# Create color array
cmap = plt.cm.tab20(np.linspace(0, 1, 15))
colors = np.array(['#CCCCCC'] * len(y_sub))
for i, cls in enumerate(sorted(top15_classes)):
    colors[y_sub == cls] = matplotlib.colors.to_hex(cmap[i])

# Unsupervised
ax = axes[0]
ax.scatter(umap_unsup[:, 0], umap_unsup[:, 1], c=colors, s=1, alpha=0.5, rasterized=True)
ax.set_xlabel('UMAP 1', fontsize=11)
ax.set_ylabel('UMAP 2', fontsize=11)
ax.set_title('Unsupervised VAE Embeddings', fontsize=12, fontweight='bold')

# Semi-supervised
ax = axes[1]
ax.scatter(umap_semisup[:, 0], umap_semisup[:, 1], c=colors, s=1, alpha=0.5, rasterized=True)
ax.set_xlabel('UMAP 1', fontsize=11)
ax.set_ylabel('UMAP 2', fontsize=11)
ax.set_title('Semi-supervised VAE Embeddings', fontsize=12, fontweight='bold')

# Legend
handles = []
for i, cls in enumerate(sorted(top15_classes)):
    name = label_encoder.inverse_transform([cls])[0][:20]
    handles.append(plt.Line2D([0], [0], marker='o', color='w',
                               markerfacecolor=matplotlib.colors.to_hex(cmap[i]),
                               markersize=8, label=name))

fig.legend(handles=handles, loc='center right', bbox_to_anchor=(1.12, 0.5), fontsize=9)

plt.suptitle('UMAP Visualization of VAE Latent Space (colored by lithology)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig_umap_lithology.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'fig_umap_lithology.png'}")

## 13. Network Diagrams

In [ ]:
def draw_vae_architecture(ax, model_type='unsupervised', n_classes=None):
    """
    Draw VAE architecture diagram.
    
    Parameters
    ----------
    ax : matplotlib axis
    model_type : str
        'unsupervised' or 'semi-supervised'
    n_classes : int
        Number of classes (for semi-supervised)
    """
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 6)
    ax.set_aspect('equal')
    ax.axis('off')
    
    # Colors
    input_color = '#90EE90'   # Light green
    hidden_color = '#87CEEB'  # Sky blue
    latent_color = '#FFD700'  # Gold
    output_color = '#FFA07A'  # Light salmon
    class_color = '#DDA0DD'   # Plum
    
    # Layer positions (x)
    x_input = 1
    x_enc1 = 2.5
    x_enc2 = 4
    x_latent = 5.5
    x_dec1 = 7
    x_dec2 = 8.5
    x_output = 10
    
    # Main path y-position
    y_main = 3
    
    # Draw boxes
    def draw_box(x, y, w, h, color, text, fontsize=9):
        rect = plt.Rectangle((x - w/2, y - h/2), w, h, 
                             facecolor=color, edgecolor='black', linewidth=1.5)
        ax.add_patch(rect)
        ax.text(x, y, text, ha='center', va='center', fontsize=fontsize, fontweight='bold')
    
    # Draw arrow
    def draw_arrow(x1, y1, x2, y2, style='-'):
        ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                   arrowprops=dict(arrowstyle='->', color='black', lw=1.5, ls=style))
    
    # Input
    draw_box(x_input, y_main, 0.8, 1.2, input_color, 'Input\n(6)')
    
    # Encoder
    draw_box(x_enc1, y_main, 0.8, 1.2, hidden_color, 'FC\n64\nBN')
    draw_box(x_enc2, y_main, 0.8, 1.2, hidden_color, 'FC\n32\nBN')
    
    # Latent (mu and logvar)
    draw_box(x_latent, y_main + 0.7, 0.8, 0.8, latent_color, 'μ (10)')
    draw_box(x_latent, y_main - 0.7, 0.8, 0.8, latent_color, 'σ² (10)')
    
    # Decoder
    draw_box(x_dec1, y_main, 0.8, 1.2, hidden_color, 'FC\n32\nBN')
    draw_box(x_dec2, y_main, 0.8, 1.2, hidden_color, 'FC\n64\nBN')
    
    # Output
    draw_box(x_output - 0.5, y_main, 0.8, 1.2, output_color, 'Output\n(6)')
    
    # Arrows - encoder
    draw_arrow(x_input + 0.4, y_main, x_enc1 - 0.4, y_main)
    draw_arrow(x_enc1 + 0.4, y_main, x_enc2 - 0.4, y_main)
    draw_arrow(x_enc2 + 0.4, y_main + 0.2, x_latent - 0.4, y_main + 0.7)
    draw_arrow(x_enc2 + 0.4, y_main - 0.2, x_latent - 0.4, y_main - 0.7)
    
    # Sampling (z)
    ax.text(x_latent + 0.6, y_main, 'z', fontsize=12, fontweight='bold', ha='center', va='center')
    draw_arrow(x_latent + 0.4, y_main + 0.5, x_latent + 0.6, y_main + 0.1)
    draw_arrow(x_latent + 0.4, y_main - 0.5, x_latent + 0.6, y_main - 0.1)
    
    # Arrows - decoder
    draw_arrow(x_latent + 0.8, y_main, x_dec1 - 0.4, y_main)
    draw_arrow(x_dec1 + 0.4, y_main, x_dec2 - 0.4, y_main)
    draw_arrow(x_dec2 + 0.4, y_main, x_output - 0.5 - 0.4, y_main)
    
    # Classification head for semi-supervised
    if model_type == 'semi-supervised' and n_classes:
        y_class = 1.2
        draw_box(x_dec1, y_class, 0.8, 0.8, class_color, 'FC\n64')
        draw_box(x_dec2, y_class, 0.8, 0.8, class_color, f'FC\n{n_classes}')
        
        # Arrows from z to classifier
        draw_arrow(x_latent + 0.6, y_main - 0.3, x_dec1 - 0.4, y_class)
        draw_arrow(x_dec1 + 0.4, y_class, x_dec2 - 0.4, y_class)
        
        # Label
        ax.text(x_dec2 + 0.8, y_class, 'Classes', fontsize=10, ha='left', va='center')
    
    # Labels
    ax.text(x_enc1 + 0.75, y_main + 1.5, 'Encoder', fontsize=11, ha='center', fontweight='bold')
    ax.text(x_dec1 + 0.75, y_main + 1.5, 'Decoder', fontsize=11, ha='center', fontweight='bold')
    ax.text(x_latent, y_main + 1.8, 'Latent Space', fontsize=11, ha='center', fontweight='bold')

In [ ]:
# Draw both architectures
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Unsupervised
ax = axes[0]
draw_vae_architecture(ax, model_type='unsupervised')
ax.set_title('Unsupervised VAE Architecture', fontsize=14, fontweight='bold', pad=20)

# Semi-supervised
ax = axes[1]
draw_vae_architecture(ax, model_type='semi-supervised', n_classes=n_classes)
ax.set_title('Semi-Supervised VAE Architecture', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig_network_diagrams.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'fig_network_diagrams.png'}")

## 14. Summary and Export

In [ ]:
# Final summary
print("="*70)
print("BHAI VAE PAPER NOTEBOOK - SUMMARY")
print("="*70)
print(f"\nTimestamp: {datetime.now().isoformat()}")
print(f"Device: {DEVICE}")
print(f"Random seed: {SEED}")

print(f"\n--- Dataset ---")
print(f"Total samples: {len(df):,}")
print(f"Total boreholes: {df['Borehole_ID'].nunique():,}")
print(f"Expeditions: {len(expeditions)}")
print(f"Lithology classes: {n_classes}")

print(f"\n--- Model Architecture ---")
print(f"Input dim: {INPUT_DIM}")
print(f"Latent dim: {LATENT_DIM}")
print(f"Hidden dims: {HIDDEN_DIMS}")

print(f"\n--- Training ---")
print(f"Epochs: {N_EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Mask ratio: {MASK_RATIO}")
print(f"β-annealing: {BETA_START} → {BETA_END} over {BETA_ANNEAL_EPOCHS} epochs")
print(f"α-annealing: {ALPHA_START} → {ALPHA_END} over {ALPHA_ANNEAL_EPOCHS} epochs")

print(f"\n--- Results ---")
print(f"Unsupervised VAE:")
print(f"  R² (overall): {r2_overall_unsup:.4f}")
print(f"  SVM Accuracy: {acc_svm:.4f}")
print(f"\nSemi-supervised VAE:")
print(f"  R² (overall): {r2_overall_semisup:.4f}")
print(f"  Classification Accuracy: {acc_semisup:.4f}")

print(f"\n--- Generated Files ---")
print(f"Models:")
for f in MODEL_DIR.glob('*.pt'):
    print(f"  {f.name}")
print(f"\nFigures:")
for f in sorted(FIGURES_DIR.glob('fig_*.png')):
    print(f"  {f.name}")

print("\n" + "="*70)
print("COMPLETE")
print("="*70)